# Реализуйте задания

1. Расширте класс для работы с матрицами, загрузкой и сохранением. Придумаейте тесты.

In [1]:
import pickle

import numpy as np

class Matrix:
    def __init__(self, rows):
        self.rows = rows
        self.num_rows = len(rows)
        self.num_cols = len(rows[0]) if rows else 0

    # def __str__(self):
    #     matrix_str = ""
    #     for row in self.rows:
    #         matrix_str += "  ".join(map(str, row)) + "\n"
    #     return matrix_str

    def __add__(self, other):
        """сложение матриц"""
        if self.num_rows != other.num_rows or self.num_cols != other.num_cols:
            raise ValueError("Матрицы должны быть одного размера")
        result = []
        for i in range(self.num_rows):
            row = []
            for j in range(self.num_cols):
                row.append(self.rows[i][j] + other.rows[i][j])
            result.append(row)
        return Matrix(result)

    def __sub__(self, other):
        """вычитание матриц"""
        if self.num_rows != other.num_rows or self.num_cols != other.num_cols:
            raise ValueError("Матрицы должны быть одного размера")
        result = []
        for i in range(self.num_rows):
            row = []
            for j in range(self.num_cols):
                row.append(self.rows[i][j] - other.rows[i][j])
            result.append(row)
        return Matrix(result)

    def __mul__(self, other):   # Cji = Aik * Bkj
        """произведение матриц"""
        if self.num_cols != other.num_rows:
            raise ValueError("Количество столбцов в первой матрице должно быть равно количеству строк во второй матрице.")
        result = []
        for i in range(self.num_rows):
            row = []
            for j in range(other.num_cols):
                element = 0
                for k in range(self.num_cols):
                    element += self.rows[i][k] * other.rows[k][j]
                row.append(element)
            result.append(row)
        return Matrix(result)

    def transpose(self):
        """транспонирование матрицы: - первая строка матрицы А становится первым столбцом матрицы В"""
        transposed = [[self.rows[j][i] for j in range(self.num_rows)] for i in range(self.num_cols)]
        return Matrix(transposed)

    def determinant(self):
        """определитель матрицы, способ рекурсивного поиска"""
        if self.num_rows != self.num_cols:
            raise ValueError("Определитель можно вычислить только для квадратных матриц.")
        if self.num_rows == 1:
            return self.rows[0][0]
        if self.num_rows == 2:
            return self.rows[0][0] * self.rows[1][1] - self.rows[0][1] * self.rows[1][0]
        det = 0
        for j in range(self.num_cols):
            minor = [row[:j] + row[j + 1:] for row in self.rows[1:]]
            det += ((-1) ** j) * self.rows[0][j] * Matrix(minor).determinant()
        return det

    def inverse(self):
        """Вычисление обратной матрицы с помощью NumPy"""
        if self.num_rows != self.num_cols:
            raise ValueError("Можно вычислить только для квадратных матриц")

        # Преобразуем список списков в массив NumPy
        matrix_np = np.array(self.rows)

        # Проверяем, является ли матрица сингулярной
        if np.linalg.det(matrix_np) == 0:
            raise ValueError("Матрица сингулярна, ее нельзя вычислить обратно")

        # Вычисляем обратную матрицу с помощью NumPy
        inverse_matrix_np = np.linalg.inv(matrix_np)

        # Преобразуем обратную матрицу из массива NumPy обратно в список списков
        inverse_matrix = inverse_matrix_np.tolist()
        return Matrix(inverse_matrix)

    def save_to_file(self, file_name):
        """
        сохраняем объект в файл с помощью сериализиции
        (преобразования объекта в поток байтов)
        """
        with open(file_name, 'wb') as f:
            pickle.dump(self, f)
            
    @staticmethod
    def load_from_file(file_name):
        """
        загружаем объект автомобиля из файла с помощью десириализации
        (восстановление объекта их потока байтов)
        """
        with open(file_name, 'rb') as f:
            return pickle.load(f)
    

In [2]:
def test_save_load():
    original_matrix = Matrix([[1, 2], [3, 4]])
    original_matrix.save_to_file("matrix1.pkl")
    loaded_matrix = Matrix.load_from_file("matrix1.pkl")
    assert loaded_matrix.rows == original_matrix.rows

def test_save_load_with_empty_matrix():
    original_matrix = Matrix([])
    original_matrix.save_to_file("matrix2.pkl")
    loaded_matrix = Matrix.load_from_file("matrix2.pkl")
    assert loaded_matrix.rows == original_matrix.rows

In [3]:
original_matrix = Matrix([[1, 2], [3, 4]])
original_matrix.save_to_file("matrix1.pkl")
loaded_matrix = Matrix.load_from_file("matrix1.pkl")
assert loaded_matrix.rows == original_matrix.rows

In [4]:
original_matrix = Matrix([])
original_matrix.save_to_file("matrix2.pkl")
loaded_matrix = Matrix.load_from_file("matrix2.pkl")
assert loaded_matrix.rows == original_matrix.rows

2. Расширте классы Warehouse Product Order для управления складскими запасами, загрузкой и сохранением. Придумаейте тесты.

In [ ]:
import pickle

from bank_account import BankAccount

class Product:
    def __init__(self, name, price, quantity):
        self.name = name
        self.price = price
        self.quantity = quantity

    def __str__(self):
        return f"{self.name} - {self.price} руб. - {self.quantity} шт."

class Warehouse:
    def __init__(self):
        self.products = []

    def add_product(self, product: Product):
        """метод добавления товара на склад"""
        self.products.append(product)

    def check_inventory(self):
        """вывод всех товаров на складе"""
        for product in self.products:
            print(product)

class Order:
    def __init__(self):
        self.cart_of_orders = []

    def add_item(self, item: str, quantity: int, warehouse: Warehouse):
        """Метод добавления товара в корзину с учетом товаров на складе"""
        for product in warehouse.products:
            if product.name == item and quantity <= product.quantity:
                self.cart_of_orders.append((product, quantity))
                break
        else:
            print(f"Товар '{item}' не был добавлен в корзину из-за нехватки на складе или его отсутствия.")

    def buy_item(self, warehouse: Warehouse, bank_account: BankAccount):
        """Метод покупки товара со склада с учетом средств на счете и списания товара со склада"""
        if bank_account.balance >= self.calculate_total_cost():
            for item, quantity in self.cart_of_orders:
                for product in warehouse.products:
                    if item.name == product.name:
                        if quantity <= product.quantity:
                            product.quantity -= quantity
                            bank_account.withdraw(item.price * quantity)
                            print(f"Покупка {item.name} совершена успешно.")
                        else: 
                            print(f"Вы превысили количество имеющегося товара '{item.name}' на складе.")
                        break
                else:
                    print(f"Товар '{item.name}' отсутствует на складе.")
        else:
            print("Недостаточно средств на счете.")

    def calculate_total_cost(self):
        """метод подсчета общей суммы товаров в корзине"""
        total_cost = 0
        for product, quantity in self.cart_of_orders:
            item_cost = product.price * quantity
            total_cost += item_cost
        return total_cost

    def display_my_order(self):
        """метод выводит товары в корзине и сумму покупки"""
        total_cost = self.calculate_total_cost()  
        for product, quantity in self.cart_of_orders:
            item_cost = product.price * quantity
            print(f"{product.name}: {quantity} шт - {item_cost} руб.")
        print(f"Общая стоимость покупки: {total_cost} руб.")

    def save_to_file(self, file_name):
        """
        сохраняем объект в файл с помощью сериализиции
        (преобразования объекта в поток байтов)
        """
        with open(file_name, 'wb') as f:
            pickle.dump(self, f)
            
    @staticmethod
    def load_from_file(file_name):
        """
        загружаем объект автомобиля из файла с помощью десириализации
        (восстановление объекта их потока байтов)
        """
        with open(file_name, 'rb') as f:
            return pickle.load(f)
            